<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 15px; height: 80px">

# Project 3

### Regression and Classification with the Ames Housing Data

---

You have just joined a new "full stack" real estate company in Ames, Iowa. The strategy of the firm is two-fold:
- Own the entire process from the purchase of the land all the way to sale of the house, and anything in between.
- Use statistical analysis to optimize investment and maximize return.

The company is still small, and though investment is substantial the short-term goals of the company are more oriented towards purchasing existing houses and flipping them as opposed to constructing entirely new houses. That being said, the company has access to a large construction workforce operating at rock-bottom prices.

This project uses the [Ames housing data recently made available on kaggle](https://www.kaggle.com/c/house-prices-advanced-regression-techniques).

In [1]:
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

sns.set_style('whitegrid')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 1. Estimating the value of homes from fixed characteristics.

---

Your superiors have outlined this year's strategy for the company:
1. Develop an algorithm to reliably estimate the value of residential houses based on *fixed* characteristics.
2. Identify characteristics of houses that the company can cost-effectively change/renovate with their construction team.
3. Evaluate the mean dollar value of different renovations.

Then we can use that to buy houses that are likely to sell for more than the cost of the purchase plus renovations.

Your first job is to tackle #1. You have a dataset of housing sale data with a huge amount of features identifying different aspects of the house. The full description of the data features can be found in a separate file:

    housing.csv
    data_description.txt
    
You need to build a reliable estimator for the price of the house given characteristics of the house that cannot be renovated. Some examples include:
- The neighborhood
- Square feet
- Bedrooms, bathrooms
- Basement and garage space

and many more. 

Some examples of things that **ARE renovate-able:**
- Roof and exterior features
- "Quality" metrics, such as kitchen quality
- "Condition" metrics, such as condition of garage
- Heating and electrical components

and generally anything you deem can be modified without having to undergo major construction on the house.

---

**Your goals:**
1. Perform any cleaning, feature engineering, and EDA you deem necessary.
- Be sure to remove any houses that are not residential from the dataset.
- Identify **fixed** features that can predict price.
- Train a model on pre-2010 data and evaluate its performance on the 2010 houses.
- Characterize your model. How well does it perform? What are the best estimates of price?

> **Note:** The EDA and feature engineering component to this project is not trivial! Be sure to always think critically and creatively. Justify your actions! Use the data description file!

In [2]:
# Load the data
house = pd.read_csv('./housing.csv')

In [3]:
#residential houses
residential = ['FV','RH','RL','RP','RM']
#filter residential houses
house = house[house.MSZoning.isin(residential)]

In [4]:
house.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1450 entries, 0 to 1459
Data columns (total 81 columns):
Id               1450 non-null int64
MSSubClass       1450 non-null int64
MSZoning         1450 non-null object
LotFrontage      1191 non-null float64
LotArea          1450 non-null int64
Street           1450 non-null object
Alley            89 non-null object
LotShape         1450 non-null object
LandContour      1450 non-null object
Utilities        1450 non-null object
LotConfig        1450 non-null object
LandSlope        1450 non-null object
Neighborhood     1450 non-null object
Condition1       1450 non-null object
Condition2       1450 non-null object
BldgType         1450 non-null object
HouseStyle       1450 non-null object
OverallQual      1450 non-null int64
OverallCond      1450 non-null int64
YearBuilt        1450 non-null int64
YearRemodAdd     1450 non-null int64
RoofStyle        1450 non-null object
RoofMatl         1450 non-null object
Exterior1st      1450 non-n

In [5]:
# check mean and median value of LotFrontage,
# if they are similiar, then I can impute the NaN
# values with either median and mean

print(house.LotFrontage.median())
print(house.LotFrontage.mean())

70.0
70.0528967254408


In [6]:
#impute LotFrontage NaN values with 
# median of LotFrontage

house.LotFrontage.fillna(house.LotFrontage.median(),inplace=True)

In [7]:
#fill Nan values with 'NA' string
house.Alley.fillna('NA',inplace=True)

In [8]:
# look at NaN fields in MasVnrType and MasVnrArea
house[house.MasVnrType.isnull()].loc[:,'Exterior1st':'Foundation']

,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation
234,VinylSd,VinylSd,NaN,NaN,Gd,TA,PConc
529,Wd Sdng,Stone,NaN,NaN,Gd,TA,PConc
650,CemntBd,CmentBd,NaN,NaN,Gd,TA,PConc
936,VinylSd,VinylSd,NaN,NaN,Gd,TA,PConc
973,CemntBd,CmentBd,NaN,NaN,Gd,TA,PConc
977,VinylSd,VinylSd,NaN,NaN,Gd,TA,PConc
1243,VinylSd,VinylSd,NaN,NaN,Ex,TA,PConc
1278,VinylSd,VinylSd,NaN,NaN,Gd,TA,PConc


In [9]:
#fill MasVnrType NaN values with 'None'
house.MasVnrType.fillna('None',inplace=True)

#fill MasVnrArea NaN values with 0
house.MasVnrArea.fillna(0,inplace=True)

#since the basement related quality and ratings are 
# NaN for houses with no basement, 
# I changed the NaN values to 'NA' string
house.loc[:,'BsmtQual':'BsmtFinType2'] = house.loc[:,
                    'BsmtQual':'BsmtFinType2'].apply(lambda x: x.fillna('NA'))

In [10]:
#check values of Electrical
house.Electrical.value_counts(dropna=False)

SBrkr    1328
FuseA      90
FuseF      27
FuseP       3
Mix         1
NaN         1
Name: Electrical, dtype: int64

In [11]:
# since there is only 1 NaN value, I replaced it with the highest count,
#which is SBrkr
house.Electrical.fillna('SBrkr',inplace=True)

In [12]:
#check number of counts of houses with 0 fireplace
house[house.FireplaceQu.isnull()].loc[:,'Fireplaces'].value_counts()

0    681
Name: Fireplaces, dtype: int64

In [13]:
#check that number tallies with number of NaN values for FireplaceQu
house[house.FireplaceQu.isnull()].loc[:,'FireplaceQu'].value_counts(dropna=False)

NaN    681
Name: FireplaceQu, dtype: int64

In [14]:
#since FireplaceQu is NaN for houses with 0 fireplace, I will replace the NaN 
# with 'NA'
house.FireplaceQu.fillna('NA',inplace=True)

In [15]:
#check garage-related data for NaN values are the same
# if they are the same, it means that the houses do not have a garage
house.loc[:,'GarageType':'GarageCond'].isnull().sum()

GarageType      79
GarageYrBlt     79
GarageFinish    79
GarageCars       0
GarageArea       0
GarageQual      79
GarageCond      79
dtype: int64

In [16]:
#since the garage-related quality and conditions are NaN for houses with no
# garage, I changed the NaN values to 'NA' string

#I did not change the GarageYrBuilt NaN values, since the variable is 
# already a float

house.loc[:,['GarageType','GarageFinish','GarageQual','GarageCond']] = \
house.loc[:,['GarageType','GarageFinish','GarageQual','GarageCond']].apply(lambda x: x.fillna('NA'))

In [17]:
#changed misc NaN values to 'NA'

house.loc[:,'PoolArea':'MiscFeature'] = \
house.loc[:,'PoolArea':'MiscFeature'].apply(lambda x: x.fillna('NA'))

## Feature Engineering

In [18]:
# create a dummy variable to check whether house was
# remodelled before, 1 if Yes, and 0 if no

house['is_remod'] = (house.YearBuilt != house.YearRemodAdd).astype('int')

#drop YearBuilt and YearRemodAdd columns
house.drop(['YearBuilt','YearRemodAdd'],axis=1,inplace=True)

In [19]:
# #create a dictionary to change all quality and condition ratings to numbers
# cond_qual_dict = {'Ex':5 , 'Gd':4, 'TA':3, 'Fa':2, 'Po':1, 'NA':0}

# # #map with the dictionary
# house.ExterQual = house.ExterQual.map(cond_qual_dict)
# house.ExterCond = house.ExterCond.map(cond_qual_dict)
# house.BsmtQual = house.BsmtQual.map(cond_qual_dict) 
# house.BsmtCond = house.BsmtCond.map(cond_qual_dict) 
# house.HeatingQC = house.HeatingQC.map(cond_qual_dict)
# house.KitchenQual = house.KitchenQual.map(cond_qual_dict)
# house.FireplaceQu = house.FireplaceQu.map(cond_qual_dict)
# house.GarageQual = house.GarageQual.map(cond_qual_dict)
# house.GarageCond = house.GarageCond.map(cond_qual_dict)

In [20]:
#change to dummy variables
house.CentralAir = house.CentralAir.map(lambda x: 1 if x=='Y' else 0)

In [21]:
house.BsmtExposure.value_counts(dropna=False)

No    947
Av    219
Gd    134
Mn    112
NA     38
Name: BsmtExposure, dtype: int64

In [22]:
# #create a dictionary to change BsmtExposure ratings to numbers
# BsmtExposure_dict = {'Gd':4,'Av':3,'Mn':2,'No':1,'NA':0}
# #map with dictionary
# house.BsmtExposure = house.BsmtExposure.map(BsmtExposure_dict)

In [23]:
house.BsmtFinType1.value_counts(dropna=False)

Unf    424
GLQ    418
ALQ    220
BLQ    147
Rec    130
LwQ     74
NA      37
Name: BsmtFinType1, dtype: int64

In [24]:
house.BsmtFinType2.value_counts(dropna=False)

Unf    1246
Rec      54
LwQ      46
NA       38
BLQ      33
ALQ      19
GLQ      14
Name: BsmtFinType2, dtype: int64

In [25]:
# #create a dictionary to change BsmtFinType ratings to numbers
# BsmtFinType_dict = {'GLQ':6,'ALQ':5,'BLQ':4,'Rec':3,'LwQ':2,'Unf':1,'NA':0}

# #map with dictionary
# house.BsmtFinType1 = house.BsmtFinType1.map(BsmtFinType_dict)
# house.BsmtFinType2 = house.BsmtFinType2.map(BsmtFinType_dict)

In [26]:
# #create a dictionary to change Functional ratings to numbers
# Functional_dict = {'Typ':8,'Min1':7,'Min2':6,'Mod':5,'Maj1':4,'Maj2':3,
#                    'Sev':2,'Sal':1}
# #map with dictionary
# house.Functional = house.Functional.map(Functional_dict)

## Define Fixed and Renovatable Features

In [45]:
fixed = ['MSSubClass','MSZoning','LotFrontage','LotArea','Street',
         'Alley','LotShape','LandContour','LotConfig','LandSlope',
         'Neighborhood','Condition1','Condition2','BldgType','HouseStyle',
         'MasVnrType','MasVnrArea','Foundation','BsmtFinSF1','BsmtFinSF2',
         'BsmtUnfSF','TotalBsmtSF','1stFlrSF','2ndFlrSF','LowQualFinSF',
         'GrLivArea','BsmtFullBath','BsmtHalfBath','FullBath','HalfBath',
         'BedroomAbvGr','KitchenAbvGr','TotRmsAbvGrd','Fireplaces','GarageType',
         'GarageCars','GarageArea','PavedDrive','WoodDeckSF','OpenPorchSF',
         'EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea','MiscFeature',
         'MiscVal'
]

renov = ['Utilities','OverallQual','OverallCond','RoofStyle','RoofMatl',
         'Exterior1st','Exterior2nd','ExterQual','ExterCond','BsmtQual',
         'BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','Heating',
         'HeatingQC','CentralAir','Electrical','KitchenQual','Functional',
         'FireplaceQu','GarageFinish','GarageQual','GarageCond','PoolQC',
         'Fence'
]

target = ['SalePrice']

house_fixed = house[fixed]
house_renov = house[renov]

In [85]:
fixed_categorical = []
fixed_numeric = []

for column in house_fixed:
    if house_fixed[column].dtypes=='object':
        fixed_categorical.append(column)
    elif column == 'MSSubClass':
        print('Skip',column)
    else:
        fixed_numeric.append(column)

Skip MSSubClass


In [86]:
pd.get_dummies(house_fixed.MSSubClass,drop_first=True,prefix='MSSubClass').head()

,MSSubClass_30,MSSubClass_40,MSSubClass_45,MSSubClass_50,MSSubClass_60,MSSubClass_70,MSSubClass_75,MSSubClass_80,MSSubClass_85,MSSubClass_90,MSSubClass_120,MSSubClass_160,MSSubClass_180,MSSubClass_190
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [87]:
pd.get_dummies(house_fixed[fixed_categorical],drop_first=True).head()

,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Pave,Alley_NA,Alley_Pave,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_HLS,...,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_NA,PavedDrive_P,PavedDrive_Y,MiscFeature_NA,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC
0,0,1,0,1,1,0,0,0,1,0,...,0,0,0,0,0,1,1,0,0,0
1,0,1,0,1,1,0,0,0,1,0,...,0,0,0,0,0,1,1,0,0,0
2,0,1,0,1,1,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
3,0,1,0,1,1,0,0,0,0,0,...,0,0,1,0,0,1,1,0,0,0
4,0,1,0,1,1,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0


<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 2. Determine any value of *changeable* property characteristics unexplained by the *fixed* ones.

---

Now that you have a model that estimates the price of a house based on its static characteristics, we can move forward with part 2 and 3 of the plan: what are the costs/benefits of quality, condition, and renovations?

There are two specific requirements for these estimates:
1. The estimates of effects must be in terms of dollars added or subtracted from the house value. 
2. The effects must be on the variance in price remaining from the first model.

The residuals from the first model (training and testing) represent the variance in price unexplained by the fixed characteristics. Of that variance in price remaining, how much of it can be explained by the easy-to-change aspects of the property?

---

**Your goals:**
1. Evaluate the effect in dollars of the renovate-able features. 
- How would your company use this second model and its coefficients to determine whether they should buy a property or not? Explain how the company can use the two models you have built to determine if they can make money. 
- Investigate how much of the variance in price remaining is explained by these features.
- Do you trust your model? Should it be used to evaluate which properties to buy and fix up?

In [27]:
# A:

<img src="http://imgur.com/GCAf1UX.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 3. What property characteristics predict an "abnormal" sale?

---

The `SaleCondition` feature indicates the circumstances of the house sale. From the data file, we can see that the possibilities are:

       Normal	Normal Sale
       Abnorml	Abnormal Sale -  trade, foreclosure, short sale
       AdjLand	Adjoining Land Purchase
       Alloca	Allocation - two linked properties with separate deeds, typically condo with a garage unit	
       Family	Sale between family members
       Partial	Home was not completed when last assessed (associated with New Homes)
       
One of the executives at your company has an "in" with higher-ups at the major regional bank. His friends at the bank have made him a proposal: if he can reliably indicate what features, if any, predict "abnormal" sales (foreclosures, short sales, etc.), then in return the bank will give him first dibs on the pre-auction purchase of those properties (at a dirt-cheap price).

He has tasked you with determining (and adequately validating) which features of a property predict this type of sale. 

---

**Your task:**
1. Determine which features predict the `Abnorml` category in the `SaleCondition` feature.
- Justify your results.

This is a challenging task that tests your ability to perform classification analysis in the face of severe class imbalance. You may find that simply running a classifier on the full dataset to predict the category ends up useless: when there is bad class imbalance classifiers often tend to simply guess the majority class.

It is up to you to determine how you will tackle this problem. I recommend doing some research to find out how others have dealt with the problem in the past. Make sure to justify your solution. Don't worry about it being "the best" solution, but be rigorous.

Be sure to indicate which features are predictive (if any) and whether they are positive or negative predictors of abnormal sales.

In [28]:
# A: